In [2]:
df=pd.read_excel("C:\\Users\\adypr\\Desktop\\january.xlsx")
df.head(21)

,Account Name :,Mr. Aditya Prakash,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Address :,"S/O: Gyanendra Kumar, nehru chak, nehru",NaN,NaN,NaN,NaN,NaN
1,,"chak, chacha nehru school",NaN,NaN,NaN,NaN,NaN
2,,Sampatchak-800007,NaN,NaN,NaN,NaN,NaN
3,,Patna,NaN,NaN,NaN,NaN,NaN
4,Date :,2025-01-26 00:00:00,NaN,NaN,NaN,NaN,NaN
5,Account Number :,_00000041185122386,NaN,NaN,NaN,NaN,NaN
6,Account Description:,LOTUS SAVING BANK-ADHAR- CHQ,NaN,NaN,NaN,NaN,NaN
7,Branch :,GULZARBAGH,NaN,NaN,NaN,NaN,NaN
8,Drawing Power :,0,NaN,NaN,NaN,NaN,NaN
9,Interest Rate(% p.a.):,2.7,NaN,NaN,NaN,NaN,NaN


In [41]:
import os
import re
import pandas as pd

def process_bank_statement(file_path):
    """Processes a bank statement file, converting it if necessary and extracting relevant data."""
    
    # Check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    # Determine the file extension and convert if necessary
    file_extension = file_path.split('.')[-1]
    if file_extension == "xls":
        xlsx_file_path = file_path.replace(".xls", ".xlsx")
        try:
            df = pd.read_excel(file_path, engine='xlrd')
            df.to_excel(xlsx_file_path, index=False, engine='openpyxl')
            print(f"File successfully converted to: {xlsx_file_path}")
            file_path = xlsx_file_path
        except Exception as e:
            print(f"Error converting file: {str(e)}")
            return None

    if file_path.endswith(".xlsx"):
        try:
            df = pd.read_excel(file_path)
            df_cleaned = df.dropna(how='all')
            df_cleaned.iloc[:, 0] = df_cleaned.iloc[:, 0].astype(str)

            # Find the starting index of relevant data
            start_index = df_cleaned[df_cleaned.iloc[:, 0].str.contains('Txn Date', na=False, case=False)].index[0] + 1
            df_relevant = df_cleaned.iloc[start_index:].copy()  # Explicitly create a copy here

            # Define expected column names
            expected_columns = ['Txn Date', 'Value Date', 'Description', 'Ref No./Cheque No.', 'Debit', 'Credit', 'Balance']
            df_relevant.columns = expected_columns[:len(df_relevant.columns)]

            # Extract names from the Description column
            def extract_name(description):
                """Extracts a proper name from the given description string."""
                if not isinstance(description, str):
                    return "Unknown"

                # Debug: Print the input description
                print(f"Processing Description: {description}")

                # Pattern to extract names in formats like "/NAME/"
                pattern = r"/([A-Za-z\s]+?)/"
                match = re.search(pattern, description)
                if match:
                    print(f"Matched Name (Pattern 1): {match.group(1).strip()}")
                    return match.group(1).strip()

                # Pattern to extract names in UPI descriptions like "UPI/.../NAME /..."
                pattern_alt = r"UPI/.*?/(?P<name>[A-Za-z\s]+)\s/"
                match_alt = re.search(pattern_alt, description)
                if match_alt:
                    print(f"Matched Name (Pattern 2): {match_alt.group('name').strip()}")
                    return match_alt.group("name").strip()

                # Debug: No match found
                print("No match found for Description.")
                return "Unknown"

            # Add 'Account Name' column using the extract_name function
            df_relevant['Description'] = df_relevant['Description'].astype(str).fillna('')  # Safe to modify
            df_relevant['Account Name'] = df_relevant['Description'].apply(extract_name)  # Safe to modify

            return df_relevant
        except Exception as e:
            print(f"An error occurred while processing the file: {str(e)}")
            return None
    else:
        raise ValueError(f"Unsupported file extension: {file_extension}")

if __name__ == "__main__":
    print("Welcome to the Bank Statement Processor!")
    file_path = "C:\\Users\\adypr\\Desktop\\january.xlsx"  # Replace with your actual file path
    try:
        df = process_bank_statement(file_path)
        if df is not None:
            print(df.head(2))  # Print only the first two rows
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Welcome to the Bank Statement Processor!
Processing Description:    TO TRANSFER-UPI/DR/536705741200/JAGDISH /YESB/paytmqr4c8/UPI--
Matched Name (Pattern 1): DR
Processing Description:    TO TRANSFER-UPI/DR/500189673893/IRCTC UTS/YESB/paytm-8796/Sent--
Matched Name (Pattern 1): DR
Processing Description:    BY TRANSFER-UPI/CR/500194996539/DIVYA RANI/HDFC/divs123ran/UPI--
Matched Name (Pattern 1): CR
Processing Description:    TO TRANSFER-UPI/DR/500190326628/IRCTC UTS/YESB/paytm-8796/Sent--
Matched Name (Pattern 1): DR
Processing Description:    TO TRANSFER-UPI/DR/536777531772/HARDEV G/YESB/q980193026/UPI--
Matched Name (Pattern 1): DR
Processing Description:    TO TRANSFER-UPI/DR/500197725443/EatClub/YESB/eatclub@pt/Sent u--
Matched Name (Pattern 1): DR
Processing Description:    BY TRANSFER-UPI/CR/500157104124/VIKAS BA/SBIN/vickyshela/UPI--
Matched Name (Pattern 1): CR
Processing Description:    BY TRANSFER-UPI/CR/500157278124/SHASHIKA/FDRL/ugileshash/UPI--
Matched Name (Pattern 1): CR

In [50]:
import pandas as pd    
file_path = "C:\\Users\\adypr\\Desktop\\january.xlsx"  
df = pd.read_excel(file_path)
df_cleaned = df.dropna(how='all')
df_cleaned.iloc[:, 0] = df_cleaned.iloc[:, 0].astype(str)

            # Find the starting index of relevant data
start_index = df_cleaned[df_cleaned.iloc[:, 0].str.contains('Txn Date', na=False, case=False)].index[0] + 1
df_relevant = df_cleaned.iloc[start_index:].copy()  # Explicitly create a copy here

            # Define expected column names
expected_columns = ['Txn Date', 'Value Date', 'Description', 'Ref No./Cheque No.', 'Debit', 'Credit', 'Balance']
df_relevant.columns = expected_columns[:len(df_relevant.columns)]
df_relevant['Description']



   
   

20        TO TRANSFER-UPI/DR/536705741200/JAGDISH /YE...
21        TO TRANSFER-UPI/DR/500189673893/IRCTC UTS/Y...
22        BY TRANSFER-UPI/CR/500194996539/DIVYA RANI/...
23        TO TRANSFER-UPI/DR/500190326628/IRCTC UTS/Y...
24        TO TRANSFER-UPI/DR/536777531772/HARDEV G/YE...
                             ...                        
165       TO TRANSFER-UPI/DR/502643017325/EatClub/YES...
166       TO TRANSFER-UPI/DR/502603421769/NADAR KA/YE...
167       TO TRANSFER-UPI/DR/502638828589/CONFIRM /AI...
168       TO TRANSFER-UPI/DR/502682335333/KRISHNA /YE...
170                                                  NaN
Name: Description, Length: 150, dtype: object